In [3]:
import numpy as np
import pandas as pd

In [4]:
df1 = pd.read_excel('model/data-welding.xlsx', sheet_name='Sheet1')
df2 = pd.read_excel('model/data-welding.xlsx', sheet_name='Sheet2')

In [5]:
df1.head()

,Weld No.,V,A,S,N,Bead height (H),Bead width (W),Bead Penetration (P)
0,1,-1,-1,-1,-1,2.38,10.55,3.94
1,2,1,-1,-1,-1,2.54,12.43,5.08
2,3,-1,1,-1,-1,3.07,9.76,5.12
3,4,1,1,-1,-1,2.79,13.14,6.40
4,5,-1,-1,1,-1,2.03,10.27,3.05


In [6]:
df2.head()

,Sl.No,PARAMETERS,Units,Notation,value(-2),value(-1),value(0),value(1),value(2)
0,1,Current,Amperes,A,250,325.0,400,475.0,550
1,2,Arc Voltage,Volts,V,22,22.5,29,32.5,36
2,3,Travel speed,mm/sec,S,6,9.0,12,15.0,18
3,4,Nozzle to plate distance,Mm,N,20,23.5,27,30.5,34


In [7]:
df2 = df2.set_index('Notation')
df2

,Sl.No,PARAMETERS,Units,value(-2),value(-1),value(0),value(1),value(2)
Notation,,,,,,,,
A,1,Current,Amperes,250,325.0,400,475.0,550
V,2,Arc Voltage,Volts,22,22.5,29,32.5,36
S,3,Travel speed,mm/sec,6,9.0,12,15.0,18
N,4,Nozzle to plate distance,Mm,20,23.5,27,30.5,34


In [8]:
data = df1.copy()

In [9]:
for i in range(len(df1)):
    for col in ['V', 'A', 'S', 'N']:
        data.loc[i, col] = df2[f'value({str(int(df1[col][i]))})'][col]

In [10]:
data.head()

,Weld No.,V,A,S,N,Bead height (H),Bead width (W),Bead Penetration (P)
0,1,22.5,325,9,23.5,2.38,10.55,3.94
1,2,32.5,325,9,23.5,2.54,12.43,5.08
2,3,22.5,475,9,23.5,3.07,9.76,5.12
3,4,32.5,475,9,23.5,2.79,13.14,6.40
4,5,22.5,325,15,23.5,2.03,10.27,3.05


In [11]:
df1.head()

,Weld No.,V,A,S,N,Bead height (H),Bead width (W),Bead Penetration (P)
0,1,-1,-1,-1,-1,2.38,10.55,3.94
1,2,1,-1,-1,-1,2.54,12.43,5.08
2,3,-1,1,-1,-1,3.07,9.76,5.12
3,4,1,1,-1,-1,2.79,13.14,6.40
4,5,-1,-1,1,-1,2.03,10.27,3.05


In [12]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [13]:
X = data.iloc[:, 1:5].values
y = data.iloc[:, -3:].values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=316)

In [15]:
regressor = XGBRegressor()
regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [16]:
y_pred = regressor.predict(X_test)

In [17]:
y_pred

array([[ 3.4007323,  9.073043 ,  5.531363 ],
       [ 1.784727 , 10.306603 ,  2.8014162],
       [ 2.0280247, 10.630422 ,  3.9069653],
       [ 2.6196136, 12.711475 ,  5.769111 ],
       [ 1.8279364,  9.333222 ,  3.3405437],
       [ 2.0191631, 11.003492 ,  3.9643388],
       [ 2.123408 , 10.111834 ,  4.128504 ]], dtype=float32)

In [18]:
y_test

array([[ 3.23,  9.48,  7.31],
       [ 2.08,  9.74,  2.08],
       [ 1.96, 10.4 ,  4.34],
       [ 2.79, 13.14,  6.4 ],
       [ 2.09,  8.08,  2.6 ],
       [ 1.93, 11.4 ,  3.94],
       [ 1.98,  9.26,  3.59]])

In [19]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.8039875285328163

In [20]:
test_data = pd.read_excel('model/data-welding.xlsx', sheet_name='Sheet3')
test_data

,Weld no.,V,A,S,N,H,W,P
0,5,25.5,325,15,23.5,2.03,10.27,3.05
1,8,32.5,475,15,23.5,1.99,9.42,5.37
2,16,32.5,475,15,30.5,3.05,10.58,4.19
3,23,29.0,400,12,20.0,1.96,10.40,4.34
4,31,29.0,400,12,27.0,1.94,10.72,3.68


In [21]:
pred = regressor.predict(test_data.iloc[:, 1:5])
pred

array([[ 2.0303261, 10.2690935,  3.0497825],
       [ 1.9904281,  9.421507 ,  5.369643 ],
       [ 3.0502877, 10.578713 ,  4.192559 ],
       [ 2.0280247, 10.630422 ,  3.9069653],
       [ 2.0191631, 11.003492 ,  3.9643388]], dtype=float32)

In [22]:
actual = test_data.iloc[:, -3:]
actual

,H,W,P
0,2.03,10.27,3.05
1,1.99,9.42,5.37
2,3.05,10.58,4.19
3,1.96,10.40,4.34
4,1.94,10.72,3.68


In [23]:
percentage_error = (pred - actual) / actual * 100
percentage_error

,H,W,P
0,0.016065,-0.008827,-0.007131
1,0.021512,0.015997,-0.006644
2,0.009434,-0.012161,0.061068
3,3.470647,2.215593,-9.977759
4,4.080574,2.644518,7.726597


In [24]:
import pickle

filename = 'finalized_model.sav'
# pickle.dump(regressor, open(filename, 'wb'))

regressor.save_model(f'model/{filename}')
regressor.load_model(f'model/{filename}')
regressor.predict(X_test)

array([[ 3.4007323,  9.073043 ,  5.531363 ],
       [ 1.784727 , 10.306603 ,  2.8014162],
       [ 2.0280247, 10.630422 ,  3.9069653],
       [ 2.6196136, 12.711475 ,  5.769111 ],
       [ 1.8279364,  9.333222 ,  3.3405437],
       [ 2.0191631, 11.003492 ,  3.9643388],
       [ 2.123408 , 10.111834 ,  4.128504 ]], dtype=float32)

## Graphs

In [25]:
import plotly.express as px
fig = px.scatter(data, x="V", y=["Bead height (H)", "Bead width (W)", "Bead Penetration (P)"], trendline="lowess") 
# LOESS (locally estimated scatterplot smoothing) and LOWESS (locally weighted scatterplot smoothing)
fig.show()

In [26]:
fig = px.scatter(data, x="A", y=["Bead height (H)", "Bead width (W)", "Bead Penetration (P)"], trendline="lowess")
fig.show()

In [27]:
fig = px.scatter(data, x="S", y=["Bead height (H)", "Bead width (W)", "Bead Penetration (P)"], trendline="lowess")
fig.show()

In [28]:
fig = px.scatter(data, x="N", y=["Bead height (H)", "Bead width (W)", "Bead Penetration (P)"], trendline="lowess")
fig.show()

In [29]:
@np.vectorize
def f1(A, N):
    return regressor.predict([[0, A, 0, N]])[0][0]

@np.vectorize
def f2(S, N):
    return regressor.predict([[0, 0, S, N]])[0][0]

@np.vectorize
def f3(V, A):
    return regressor.predict([[V, A, 0, 0]])[0][1]

@np.vectorize
def f4(V, S):
    return regressor.predict([[V, 0, S, 0]])[0][1]

In [30]:
import plotly.graph_objects as go

In [31]:
x = data['A'].sort_values()
y = data['N'].sort_values()
xx, yy = np.meshgrid(x, y)
zz = f1(xx, yy)

fig = go.Figure(data =
    go.Surface(
        z=zz,
        x=x,
        y=y
    ))
fig.update_layout(scene = dict(
                    xaxis_title='A',
                    yaxis_title='N',
                    zaxis_title='H'))
fig.show()

In [32]:
x = data['S'].sort_values()
y = data['N'].sort_values()
xx, yy = np.meshgrid(x, y)
zz = f2(xx, yy)

fig = go.Figure(data =
    go.Surface(
        z=zz,
        x=x,
        y=y
    ))
fig.update_layout(scene = dict(
                    xaxis_title='S',
                    yaxis_title='N',
                    zaxis_title='H'))
fig.show()

In [33]:
x = data['V'].sort_values()
y = data['A'].sort_values()
xx, yy = np.meshgrid(x, y)
zz = f3(xx, yy)

fig = go.Figure(data =
    go.Surface(
        z=zz,
        x=x,
        y=y
    ))
fig.update_layout(scene = dict(
                    xaxis_title='V',
                    yaxis_title='A',
                    zaxis_title='W'))
fig.show()

In [34]:
x = data['V'].sort_values()
y = data['S'].sort_values()
xx, yy = np.meshgrid(x, y)
zz = f4(xx, yy)

fig = go.Figure(data =
    go.Surface(
        z=zz,
        x=x,
        y=y
    ))
fig.update_layout(scene = dict(
                    xaxis_title='V',
                    yaxis_title='S',
                    zaxis_title='W'))
fig.show()